In [5]:
pip install xgboost

     |████████████████████████████████| 1.1 MB 1.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from collections import Counter
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, cross_val_score
from sklearn.feature_selection import RFECV, SelectKBest, chi2, f_classif
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier, BaggingClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PowerTransformer, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import KNNImputer, SimpleImputer, MissingIndicator
from sklearn.naive_bayes import GaussianNB
from tqdm import tqdm
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier

- Importing files:

In [3]:
df_train = pd.read_csv(r'train.csv')

In [ ]:
df_test = pd.read_csv(r'test.csv')

In [7]:
df_train

,Patient_ID,Family_Case_ID,Severity,Name,Birthday_year,Parents or siblings infected,Wife/Husband or children infected,Medical_Expenses_Family,Medical_Tent,City,Deceased
0,1,4696,3,Miss Linda Betty,NaN,0,0,225,NaN,Santa Fe,1
1,2,21436,1,Ms. Ramona Elvira,1966.0,0,1,1663,NaN,Albuquerque,0
2,3,7273,3,Mr. Mario Vernon,1982.0,0,0,221,NaN,Santa Fe,1
3,4,8226,3,Mr. Hector Joe,1997.0,0,0,220,NaN,Santa Fe,1
4,5,19689,3,Ms. Jennie Debra,1994.0,0,0,222,NaN,Santa Fe,0
...,...,...,...,...,...,...,...,...,...,...,...
895,896,6253,3,Ms. Linda Wilcox,1998.0,1,1,344,NaN,Santa Fe,0
896,897,6483,3,Mr. Haiden Vance,2006.0,0,0,258,NaN,Santa Fe,0
897,898,981,3,Miss Anaiya Love,1990.0,0,0,214,NaN,Taos,1
898,899,16418,2,Mr. Robert Williams,1994.0,1,1,812,NaN,Santa Fe,0


In [28]:
df_train.groupby(by="Deceased").count()

,Patient_ID,Family_Case_ID,Severity,Name,Birthday_year,Parents or siblings infected,Wife/Husband or children infected,Medical_Expenses_Family,Medical_Tent,City
Deceased,,,,,,,,,,
0,347,347,347,347,295,347,347,347,131,345
1,553,553,553,553,428,553,553,553,67,553


In [26]:
a=df_train.groupby(by="Family_Case_ID").count().sort_values("Deceased")
a
#a.loc[a["Patient_ID"]!=a["Deceased"]].sort_values("Patient_ID")

,Patient_ID,Severity,Name,Birthday_year,Parents or siblings infected,Wife/Husband or children infected,Medical_Expenses_Family,Medical_Tent,City,Deceased
Family_Case_ID,,,,,,,,,,
345,1,1,1,1,1,1,1,0,1,1
16208,1,1,1,1,1,1,1,0,1,1
16235,1,1,1,1,1,1,1,0,1,1
16253,1,1,1,0,1,1,1,0,1,1
16262,1,1,1,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...
10262,6,6,6,6,6,6,6,0,6,6
16969,6,6,6,6,6,6,6,0,6,6
14502,7,7,7,0,7,7,7,0,7,7


- Splitting title, first and last names:

In [ ]:
df_train.loc[:, 'Title'] = df_train.loc[:,'Name'].apply(lambda x: re.search(r"(.+?)\s(.+?)\s(.+)", x).group(1))
df_train.loc[:, 'First_Name'] = df_train.loc[:,'Name'].apply(lambda x: re.search(r"(.+?)\s(.+?)\s(.+)", x).group(2))
df_train.loc[:, 'Last_Name'] = df_train.loc[:,'Name'].apply(lambda x: re.search(r"(.+?)\s(.+?)\s(.+)", x).group(3))

In [2]:
df_test.loc[:, 'Title'] = df_test.loc[:,'Name'].apply(lambda x: re.search(r"(.+?)\s(.+?)\s(.+)", x).group(1))
df_test.loc[:, 'First_Name'] = df_test.loc[:,'Name'].apply(lambda x: re.search(r"(.+?)\s(.+?)\s(.+)", x).group(2))
df_test.loc[:, 'Last_Name'] = df_test.loc[:,'Name'].apply(lambda x: re.search(r"(.+?)\s(.+?)\s(.+)", x).group(3))

NameError: name 'df_test' is not defined

- Creating gender:

In [6]:
gender_dictionary ={'Miss' : 'F', 'Ms.' : 'F', 'Mr.' : 'M', 'Master' : 'M'} 
df_train.loc[:,'gender'] = df_train.loc[:,'Title'].map(gender_dictionary) 

In [7]:
gender_dictionary ={'Miss' : 'F', 'Mrs.' : 'F', 'Mr.' : 'M', 'Master' : 'M'} 
df_test.loc[:,'gender'] = df_test.loc[:,'Title'].map(gender_dictionary) 

- Filling missing values:

__Medical_tent__

In [9]:
df_train.Medical_Tent = df_train.Medical_Tent.fillna('No_tent')

In [10]:
df_test.Medical_Tent = df_test.Medical_Tent.fillna('No_tent')

__Birthday_year__

In [11]:
imputer = SimpleImputer(strategy='mean')
imputer.fit(df_train.loc[:,['Birthday_year']])
df_train.loc[:,'Birthday_year'] = imputer.transform(df_train.loc[:,['Birthday_year']]).astype(int)

In [12]:
df_test.loc[:,'Birthday_year'] = imputer.transform(df_test.loc[:,['Birthday_year']]).astype(int)

__City__

In [13]:
df_train.loc[:,'City'] = df_train.loc[:,'City'].fillna('Santa Fe')

- Encoding:

In [17]:
for feature in ['Medical_Tent', 'City', 'gender']:
    encoder = LabelEncoder()
    encoder.fit(df_train.loc[:, feature])
    df_train.loc[:, feature] = encoder.transform(df_train.loc[:, feature])
    df_test.loc[:, feature] = encoder.transform(df_test.loc[:, feature])

- Creating expenses per capita from medical_expenses_family:

In [18]:
df_train.loc[:,'Expenses_per_capita'] = df_train.loc[:,'Medical_Expenses_Family']/(df_train.loc[:,'Parents or siblings infected']+df_train.loc[:,'Wife/Husband or children infected']+1)

In [19]:
df_test.loc[:,'Expenses_per_capita'] = df_test.loc[:,'Medical_Expenses_Family']/(df_test.loc[:,'Parents or siblings infected']+df_test.loc[:,'Wife/Husband or children infected']+1)

- Dropping some variables to train the model:

In [26]:
X = df_train.drop(columns=['Patient_ID','Family_Case_ID','Name','Title','First_Name','Last_Name','Deceased',
                           'Medical_Expenses_Family'])
y = df_train['Deceased']

In [27]:
X_final = df_test.drop(columns=['Patient_ID','Family_Case_ID','Name','Title','First_Name','Last_Name',
                                'Medical_Expenses_Family'])

In [49]:
pipe = Pipeline(
    [
        (
            'scaler', MinMaxScaler()
        ),
        (
            'classifier', BaggingClassifier(
                base_estimator= XGBClassifier(
                    colsample_bytree = 1.0, 
                    eta = 0.63, 
                    max_depth = 2
                ),
                n_estimators=10,
                max_features=1.0,
                bootstrap=True,
                random_state=42
            )
        )
    ]
)

In [50]:
parameter_space = {
    'scaler': [
        MinMaxScaler(), 
        MinMaxScaler(feature_range=(-1,1)),
        RobustScaler(),
        StandardScaler(),
        PowerTransformer()
    ],
    'classifier__n_estimators': list(np.arange(5,50,1)),
    'classifier__max_features': list(np.arange(2,9,1)),
    'classifier__bootstrap': [True, False],
    
}

In [51]:
gridsearch = GridSearchCV(
    pipe, 
    parameter_space, 
    cv=5, 
    verbose=2,
    n_jobs=-1,
    return_train_score=True,
    scoring='accuracy'
)

In [52]:
best_model = gridsearch.fit(X, y)

Fitting 5 folds for each of 3150 candidates, totalling 15750 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    7.7s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   30.8s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 10.9min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed: 14.8min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 17.9min
[Parallel(n_jobs=-1)]: Done 4885 tasks      | elapsed: 22.3min
[Parallel(n_jobs=-1)]: Done 5816 tasks      | elapsed: 27.7min
[Parallel(n_jobs=-1)]: Done 6829 tasks      | elapsed: 33.9min
[Parallel(n_jobs=-1)]: Done 7922 tasks      | elapsed: 40.5min
[Parallel(n_jobs=-1)]: Done 9097 tasks      | e

In [55]:
best_model.best_params_

{'classifier__bootstrap': False,
 'classifier__max_features': 7,
 'classifier__n_estimators': 9,
 'scaler': MinMaxScaler(copy=True, feature_range=(0, 1))}

In [58]:
print('Mean: ', best_model.cv_results_['mean_test_score'][best_model.best_index_])
print('Split 1: ', best_model.cv_results_['split0_test_score'][best_model.best_index_])
print('Split 2: ', best_model.cv_results_['split1_test_score'][best_model.best_index_])
print('Split 3: ', best_model.cv_results_['split2_test_score'][best_model.best_index_])
print('Split 4: ', best_model.cv_results_['split3_test_score'][best_model.best_index_])
print('Split 5: ', best_model.cv_results_['split4_test_score'][best_model.best_index_])

Mean:  0.8400000000000001
Split 1:  0.8666666666666667
Split 2:  0.8333333333333334
Split 3:  0.8444444444444444
Split 4:  0.8333333333333334
Split 5:  0.8222222222222222


In [60]:
pred_final_model_3 = best_model.predict(X_final)

In [61]:
version3 = pd.DataFrame({'Patient_ID': list(df_test.Patient_ID.values), 'Deceased':list(pred_final_model_3)}) 

In [62]:
version3.to_csv('m20190922_version3.csv', index=False)